In [1273]:
import pandas as pd
data = pd.read_csv("CTF.csv")
#data.columns = data.columns.str.strip()
#data = data.rename({'Year Missing': 'YearMissing'})
#data = data.replace({'Easy': 1})
data = data.replace({'Medium': "Hard"})
#data = data.replace({'Hard': 3})

In [1253]:
data.describe()

,Continent,Age,Year Missing,Gender,Race
count,202.000000,202.000000,202.000000,202.000000,202.000000
mean,2.202970,34.702984,1998.301980,0.376238,0.930693
std,1.618731,18.605353,16.813963,0.485644,1.148284
min,0.000000,0.002700,1951.000000,0.000000,0.000000
25%,0.000000,19.000000,1985.000000,0.000000,0.000000
50%,2.000000,35.000000,2002.500000,0.000000,0.000000
75%,4.000000,49.000000,2011.000000,1.000000,2.000000
max,5.000000,75.000000,2024.000000,1.000000,4.000000


In [1255]:
data

,Continent,Age,Year Missing,Gender,Race,Findability
0,1,27.0,2001,0,3,Hard
1,4,35.0,2001,0,2,Easy
2,0,13.0,2002,1,0,Easy
3,1,38.0,2002,1,3,Hard
4,1,41.0,2002,1,3,Hard
...,...,...,...,...,...,...
197,5,14.0,1998,1,0,Hard
198,0,1.0,1998,1,1,Hard
199,1,28.0,1998,0,3,Hard
200,1,38.0,1999,0,3,Hard


In [1337]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

X = data.drop(columns=["Findability"])
y = data["Findability"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 31)

### N.America = 0, S.America = 1, Europe = 2, Africa = 3, Asia = 4, Oceania = 5
### Male = 0, Female = 1
### W = 0, B = 1, A = 2, H = 3, N = 4
### (Continent, Age, Year, Gender, Race) ###

model = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 17, min_samples_split = 2)
model.fit(X_train.values, y_train)

#joblib.dump(model, "CTFv7.joblib")
#model = joblib.load("CTF.joblib")

# Easy, Hard, Easy, Hard, Easy, Easy, Easy, Hard, Hard, Easy
predictions = model.predict([[4, 59, 2023, 1, 2], [3, 17, 1998, 0, 2], 
                             [0, 10, 2024, 1, 0], [4, 22, 1945, 0, 2], 
                             [5, 48, 2024, 1, 0], [1, 13, 2023, 1, 4],
                             [2, 44, 2023, 0, 3], [2, 35, 2015, 0, 0],
                             [0, 17, 2015, 0, 3], [0, 25, 2023, 1, 1]])
predictions1 = model.predict(X_test)
score = accuracy_score(y_test, predictions1)
print(predictions, f"\nAccuracy: {score}")

['Easy' 'Hard' 'Easy' 'Hard' 'Easy' 'Easy' 'Easy' 'Easy' 'Easy' 'Easy'] 
Accuracy: 0.7560975609756098


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [1265]:
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization

# Define the parameter grid to tune the hyperparameters
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

X = data.drop(columns=["Findability"])
y = data["Findability"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1267]:

dtree_reg = DecisionTreeRegressor(random_state=42) # Initialize a decision tree regressor
grid_search = GridSearchCV(estimator=dtree_reg, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_dtree_reg = grid_search.best_estimator_ # Get the best estimator from the grid search
y_pred = best_dtree_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
print(f"Test RMSE: {rmse}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}
Test RMSE: 1.014114853982106


In [1269]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Define the function to optimize using cross-validation
def dtree_cv(max_depth, min_samples_split, min_samples_leaf):
    # Define the model with the parameters to be optimized
    estimator = DecisionTreeRegressor(
        max_depth=int(max_depth),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        random_state=2
    )
    cval = cross_val_score(estimator, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    return cval.mean() # The optimizer tries to maximize the function, so we negate the score to minimize it

# Define the parameter bounds
param_bounds = {
    'max_depth': (1, 20),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20)
}

optimizer = BayesianOptimization(
    f=dtree_cv,
    pbounds=param_bounds,
    random_state=1,
)

optimizer.maximize(n_iter=25, init_points=5) # Bayesian optimization
best_params_bayes = optimizer.max['params']
best_params_bayes['max_depth'] = int(best_params_bayes['max_depth'])
best_params_bayes['min_samples_split'] = int(best_params_bayes['min_samples_split'])
best_params_bayes['min_samples_leaf'] = int(best_params_bayes['min_samples_leaf'])
best_score_bayes = optimizer.max['target']

print(f"Best Parameters (Bayesian Optimization): {best_params_bayes}")
print(f"Best Score (Bayesian Optimization): {best_score_bayes}")

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 1         | -0.961    | 8.923     | 14.69     | 2.002     |
| 2         | -1.367    | 6.744     | 3.788     | 3.662     |
| 3         | -1.062    | 4.539     | 7.566     | 9.142     |
| 4         | -1.098    | 11.24     | 8.965     | 14.33     |
| 5         | -0.8973   | 4.885     | 17.68     | 2.493     |
| 6         | -0.9022   | 8.177     | 19.54     | 6.465     |
| 7         | -0.9015   | 1.15      | 17.02     | 10.71     |
| 8         | -0.9022   | 3.533     | 19.74     | 19.86     |
| 9         | -0.9022   | 19.89     | 19.52     | 2.237     |
| 10        | -0.9022   | 17.88     | 20.0      | 19.38     |
| 11        | -0.9015   | 1.184     | 8.454     | 19.98     |
| 12        | -0.9022   | 19.8      | 19.83     | 10.57     |
| 13        | -0.9209   | 2.356     | 1.237     | 19.97     |
| 14        | -1.092    | 19.39     | 1.567     | 19.64     |
| 15    

In [1115]:
from sklearn import tree

tree.export_graphviz(model, out_file="CTF_Visual.dot",
                   feature_names=["Continent", "Age", "Year Missing", "Gender", "Race"],
                   class_names=sorted(y.unique()),
                    label="all",
                    rounded=True,
                    filled=True)